In [20]:
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

In [ ]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])
coefficients[:10], bases[:10]


In [26]:
t = 1
qc = QuantumCircuit(10)

for c, b in zip(coefficients, bases):
    # implement e^{-i*c*b*t}
    q = []
    for i, op in enumerate(b):
        if (op == 'X'):
            qc.h(i)
            q.append(i)
        elif (op == 'Z'):
            q.append(i)
        elif (op == 'Y'):
            qc.h(i)
            qc.s(i)
            q.append(i)
        elif (op == 'I'):
            continue
        else:
            print(op, "case not accounted for")
    if (len(q) > 0):
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        qc.rz(2*c*t, q[-1])
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
            elif (op == 'Z'):
                continue
            elif (op == 'Y'):
                qc.sdg(i)
                qc.h(i)
            elif (op == 'I'):
                continue
            else:
                print("case not accounted for")

In [27]:
qc.draw()

┌───┐┌───┐                                                               »
q_0: ┤ H ├┤ S ├───────────────────────────────────────────────────────────────»
     └───┘└───┘                                                               »
q_1: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
q_2: ─────────────────────────────────────────────────────────────────────────»
     ┌───┐┌───┐                                     ┌─────┐ ┌───┐  ┌───┐      »
q_3: ┤ H ├┤ S ├──■───────────────────────────────■──┤ Sdg ├─┤ H ├──┤ H ├──────»
     ├───┤├───┤  │                               │  └─────┘┌┴───┴┐ ├───┤ ┌───┐»
q_4: ┤ H ├┤ S ├──┼────■──────────────────────────┼─────■───┤ Sdg ├─┤ H ├─┤ H ├»
     └───┘└───┘  │    │                          │     │   └─────┘ └───┘ └───┘»
q_5: ────────────┼────┼──────────────────────────┼─────┼──────────────────────»
                 │    │                          │     │                      »
q_6: ────────────┼────┼──────────────────────────┼─────┼──────────────────────»
                 │    │                          │     │                      »
q_7: ────────────┼────┼──────────────────────────┼─────┼──────────────────────»
     ┌───┐┌───┐  │    │                          │     │          ┌─────┐┌───┐»
q_8: ┤ H ├┤ S ├──┼────┼────■─────────────────────┼─────┼──────■───┤ Sdg ├┤ H ├»
     ├───┤├───┤┌─┴─┐┌─┴─┐┌─┴─┐┌───────────────┐┌─┴─┐ ┌─┴─┐  ┌─┴─┐ ├─────┤├───┤»
q_9: ┤ H ├┤ S ├┤ X ├┤ X ├┤ X ├┤ Rz(0.0060693) ├┤ X ├─┤ X ├──┤ X ├─┤ Sdg ├┤ H ├»
     └───┘└───┘└───┘└───┘└───┘└───────────────┘└───┘ └───┘  └───┘ └─────┘└───┘»
«                                                                          »
«q_0: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«q_1: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«q_2: ─────────────────────────────────────────────────────────────────────»
«                                                    ┌───┐┌───┐ ┌───┐      »
«q_3: ────────────■───────────────────────────────■──┤ H ├┤ H ├─┤ S ├──────»
«                 │                               │  └───┘├───┤ ├───┤ ┌───┐»
«q_4: ────────────┼────■──────────────────────────┼────■──┤ H ├─┤ H ├─┤ S ├»
«                 │    │                          │    │  └───┘ └───┘ └───┘»
«q_5: ────────────┼────┼──────────────────────────┼────┼───────────────────»
«                 │    │                          │    │                   »
«q_6: ────────────┼────┼──────────────────────────┼────┼───────────────────»
«                 │    │                          │    │                   »
«q_7: ────────────┼────┼──────────────────────────┼────┼───────────────────»
«     ┌───┐┌───┐  │    │                          │    │       ┌─────┐┌───┐»
«q_8: ┤ H ├┤ S ├──┼────┼────■─────────────────────┼────┼────■──┤ Sdg ├┤ H ├»
«     ├───┤├───┤┌─┴─┐┌─┴─┐┌─┴─┐┌───────────────┐┌─┴─┐┌─┴─┐┌─┴─┐├─────┤├───┤»
«q_9: ┤ H ├┤ S ├┤ X ├┤ X ├┤ X ├┤ Rz(0.0060693) ├┤ X ├┤ X ├┤ X ├┤ Sdg ├┤ H ├»
«     └───┘└───┘└───┘└───┘└───┘└───────────────┘└───┘└───┘└───┘└─────┘└───┘»
«                                                                            »
«q_0: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«q_1: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«q_2: ───────────────────────────────────────────────────────────────────────»
«                                               ┌─────┐ ┌───┐ ┌───┐          »
«q_3: ───────■───────────────────────────────■──┤ Sdg ├─┤ H ├─┤ H ├───────■──»
«            │                               │  └─────┘┌┴───┴┐├───┤┌───┐  │ 